In [203]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [204]:
 url="https://pes.cnbv.gob.mx/Consulta/ConsultaEntidad?entidad=&nombreCorto=&subsectorName=NULL&Buscar=&DXScript=1_145%2C1_81%2C1_80%2C14_25%2C14_2%2C1_137%2C1_130%2C1_88%2C14_7%2C1_78%2C1_128%2C1_90%2C1_89%2C14_8%2C1_143%2C1_114%2C1_144%2C1_109%2C14_9%2C1_136%2C1_135%2C1_121%2C14_24%2C1_131%2C1_85%2C1_110%2C1_99%2C1_106%2C1_140%2C1_118%2C1_120%2C14_15%2C1_129%2C1_123%2C14_16%2C14_18%2C1_127%2C1_134%2C1_138%2C14_21%2C14_23%2C1_87%2C5_5%2C5_4%2C4_11%2C4_10%2C4_6%2C4_7%2C4_9%2C14_12%2C4_5%2C1_98%2C1_101%2C4_12%2C4_13%2C1_97%2C1_91%2C1_139%2C1_115%2C14_11%2C1_126%2C1_133%2C7_51%2C1_83%2C7_53%2C14_17%2C1_92%2C1_95%2C1_102%2C14_0%2C1_105%2C1_93%2C14_1%2C1_94%2C14_3%2C1_96%2C1_107%2C14_5%2C1_119%2C1_104%2C14_13%2C14_14%2C1_103%2C1_108%2C10_2%2C10_1%2C10_3%2C10_4%2C14_4%2C9_1%2C9_5%2C14_19%2C9_4%2C8_10%2C8_17%2C8_24%2C8_26%2C8_9%2C8_12%2C8_13%2C8_18%2C14_20%2C8_21%2C8_23%2C8_22%2C8_16%2C8_19%2C8_20%2C8_14%2C8_15%2C8_25%2C8_11%2C6_12%2C14_22"


In [205]:
#Request get con tiempo de espera
r = requests.get(url, verify=False)

c:\Users\claud\miniconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pes.cnbv.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [206]:
#Parseo de la pagina
soup = BeautifulSoup(r.text, 'html.parser')

In [207]:
#obtener la informacion de la tabla
table = soup.find('table', attrs={'class':'dxgvControl_Office2010Silver'})

In [208]:
#Limpiar información
rows = table.find_all('tr')
data = [[ele.text.strip() for ele in row.find_all('td')] for row in rows[1:]]
#Convertir a dataframe
df = pd.DataFrame(data)
#Eliminar 14 primeras filas
df = df.drop(df.index[0:15])
df = df[df[4].notna()]
df = df[df[4] != ""]
#Dejar 5 columnas
df = df.iloc[:, 0:4]
#Renombrar columnas
df.columns = ["casfim", "razon_social", "nombre_corto", "sector"]

In [209]:
df

,casfim,razon_social,nombre_corto,sector
15,053383,"+TASA ACTIVO, S.A DE C.V. FONDO DE INVERSIÓN D...",+TASAA1,FONDOS DE INVERSIÓN DE RENTA VARIABLE
16,053382,"+TASA LARGO PLAZO, S.A. DE C.V. FONDO DE INVER...",+TASALP,FONDOS DE INVERSIÓN EN INSTRUMENTOS DE DEUDA
17,053650,"+TASA MX, S.A. DE C.V. FONDO DE INVERSIÓN EN I...",+TASAMX,FONDOS DE INVERSIÓN EN INSTRUMENTOS DE DEUDA
18,053384,"+TASA PREVISIONAL, S.A. DE C.V. FONDO DE INVER...",+TASAPR,FONDOS DE INVERSIÓN EN INSTRUMENTOS DE DEUDA
19,053381,"+TASA US, S.A. DE C.V. FONDO DE INVERSIÓN EN I...",+TASAUS,FONDOS DE INVERSIÓN EN INSTRUMENTOS DE DEUDA
...,...,...,...,...
5166,,"ZUKZA CENTRO CAMBIARIO, S.A. DE C.V.",CC 20480,CENTROS CAMBIARIOS
5167,0695414,"ZUMPT, S.A. DE C.V., SOFOM, E.N.R.",,"SOCIEDADES FINANCIERAS DE OBJETO MÚLTIPLE, ENT..."
5168,0690592,"ZURITA CONSULTORES FINANCIEROS, S.A. DE C.V., ...",,"SOCIEDADES FINANCIERAS DE OBJETO MÚLTIPLE, ENT..."
5169,,"ZURITA Y MARRÓN ASESORES INDEPENDIENTES, S.A. ...",ZURITA Y MARRÓN,ASESORES EN INVERSIÓN PERSONAS MORALES


In [210]:
#Salvar en excel
df.to_excel("D:/padron_ent_cnbv.xlsx")